In [1]:
import tensorflow as tf
import numpy as np

In [12]:
# Load MNIST dataset
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
X_train = mnist.train.images  # Returns np.array
y_train = np.asarray(mnist.train.labels, dtype=np.int32)
X_test = mnist.test.images  # Returns np.array
y_test = np.asarray(mnist.test.labels, dtype=np.int32)

to_categorical = tf.keras.utils.to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [13]:
y_test.shape

(10000, 10)

In [14]:
def model(x, logits=False, training=False):
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    # MNIST images are 28x28 pixels, and have one color channel
    input_layer = tf.reshape(x, [-1, 28, 28, 1])

    # Convolutional Layer #1
    # Computes 32 features using a 5x5 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 28, 28, 1]
    # Output Tensor Shape: [batch_size, 28, 28, 32]
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 28, 28, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2
    # Computes 64 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 14, 14, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 64]
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 14, 14, 64]
    # Output Tensor Shape: [batch_size, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 7, 7, 64]
    # Output Tensor Shape: [batch_size, 7 * 7 * 64]
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 7 * 7 * 64]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=training)

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    logits_ = tf.layers.dense(inputs=dropout, units=10)
    y = tf.nn.softmax(logits_, name="softmax_tensor")
    
    if logits:
        return y, logits_
    return y

class Dummy:
    pass

env = Dummy()
env.x = tf.placeholder(tf.float32, (None, 784), name='x')
env.y = tf.placeholder(tf.float32, (None, 10), name='y')
env.training = tf.placeholder_with_default(False, (), name='mode')
env.ybar, logits = model(env.x, logits=True, training=env.training)
count = tf.equal(tf.argmax(env.y, axis=1), tf.argmax(env.ybar, axis=1))
env.acc = tf.reduce_mean(tf.cast(count, tf.float32), name='acc')
xent = tf.nn.softmax_cross_entropy_with_logits(labels=env.y, logits=logits)
env.loss = tf.reduce_mean(xent, name='loss')
optimizer = tf.train.AdamOptimizer()
env.train_op = optimizer.minimize(env.loss)
env.saver = tf.train.Saver()

print('Initializing graph')

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

                                               
def evaluate(sess, env, X_data, y_data, batch_size=128):
    print('Evaluating')
    loss, acc = sess.run([env.loss, env.acc], feed_dict={env.x: X_data, env.y: y_data})
    print(' loss: {0:.4f} acc: {1:.4f}'.format(loss, acc))
    return loss, acc

def train(sess, env, X_data, y_data, X_valid=None, y_valid=None, epochs=1, load=False, shuffle=False, batch_size=128, name='model'):

    if load:
        if not hasattr(env, 'saver'):
            print('Error: cannot find saver op')
            return
        print('Loading saved model')
        return env.saver.restore(sess, 'model/{}'.format(name))

    print('Train model')
    for epoch in range(epochs):
        print('\nEpoch {0}/{1}'.format(epoch + 1, epochs))
        sess.run(env.train_op, feed_dict={env.x: X_data,
                                              env.y: y_data,
                                              env.training: True})

    if hasattr(env, 'saver'):
        print('Saving model')
#         os.mkdir('model')
        env.saver.save(sess, 'model/{}'.format(name))
                                               
def predict(sess, env, X_data):
    print('Predicting')
    yval = sess.run(env.ybar, feed_dict={env.x: X_data})
    return yval

print('Training')

train(sess, env, X_train, y_train, load=False, epochs=1, name='mnist')

print('Evaluating on clean data')

evaluate(sess, env, X_test, y_test)

Initializing graph
Training
Train model

Epoch 1/1
Saving model
Evaluating on clean data
Evaluating
 loss: 2.0750 acc: 0.2530


(2.075032, 0.253)

In [16]:
predict(sess, env, X_test[:2])

Predicting


array([[0.09406996, 0.06860119, 0.09676023, 0.13229975, 0.0853923 ,
        0.07464116, 0.081567  , 0.14259279, 0.11682194, 0.10725367],
       [0.10500295, 0.06399569, 0.13599719, 0.17741226, 0.0686805 ,
        0.07716009, 0.10072666, 0.07264616, 0.12114762, 0.07723088]],
      dtype=float32)